In [3]:
from constant import RASA_NLU_MODEL_PATH
from rasa.nlu.model import Interpreter

import requests
import re
import pytz
import datetime as dt
import string
import json

import pandas as pd
import numpy as np

In [15]:
REGEX_DATE = r"(3[01]|[12][0-9]|0?[1-9])[-\/:|](1[0-2]|0?[1-9])([-\/:|](2[0-1][0-9][0-9]))"
REGEX_DAY_MONTH = r"(3[01]|[12][0-9]|0?[1-9])[-\/:|](1[0-2]|0?[1-9])"
REGEX_MONTH_YEAR = r"(1[0-2]|0?[1-9])([-\/:|](2[0-1][0-9][0-9]))"

def parse_datetime(msg, timezone="Asia/Ho_Chi_Minh"):
    try:  
        tz = pytz.timezone(timezone)
        now = dt.datetime.now(tz=tz)

        date_str = []
        regex = REGEX_DATE
        regex_day_month = REGEX_DAY_MONTH
        regex_month_year = REGEX_MONTH_YEAR
        pattern = re.compile("(%s|%s|%s)" % (
            regex, regex_month_year, regex_day_month), re.UNICODE)

        matches = pattern.finditer(msg)
        for match in matches:
            _dt = match.group(0)
            _dt = _dt.replace("/", "-").replace("|", "-").replace(":", "-").replace("tháng", "-").replace("thang", "-")
            for i in range(len(_dt.split("-"))):
                if len(_dt.split("-")[i]) == 1:
                    _dt = _dt.replace(_dt.split("-")[i], "0"+_dt.split("-")[i])
            if len(_dt.split("-")) == 2:
                pos1 = _dt.split("-")[0]
                pos2 = _dt.split("-")[1]
                if 0 < int(pos1) < 32 and 0 < int(pos2) < 13:
                    _dt = pos1+"-"+pos2+"-"+str(now.year)
            date_str.append(_dt)

        if(len(date_str) != 1):
            msg = msg.replace("/", "-").replace("|", "-").replace(":", "-").replace("tháng", "-").replace("thang", "-").replace("sáng", "9:00").replace("sang", "9:00").replace("chiều", "15:00").replace("chieu", "15:00")
            interpreter = Interpreter.load(RASA_NLU_MODEL_PATH)
            parsed = interpreter.parse(msg)
            duckling_entity = [i for i in parsed['entities'] if i['extractor'] == 'DucklingEntityExtractor']
            
            if len(duckling_entity) == 1:
                if type(duckling_entity[0]['value']) == str:
                    datetime_parsed = dt.datetime.strptime(duckling_entity[0]['value'][:-6],'%Y-%m-%dT%H:%M:%S.%f')
                else:
                    datetime_parsed = -1
            else:
                datetime_parsed = -1
        else:
            datetime_parsed = dt.datetime.strptime(date_str[0],'%d-%m-%Y')
    except Exception as e:
        datetime_parsed = -1
        print(e)
    finally:
        return datetime_parsed

def get_day_of_week(msg):
    datetime_parsed = parse_datetime(msg)
    if datetime_parsed == -1:        
        day_of_week = -1
    else:
        day_of_week = datetime_parsed.weekday()
    return day_of_week

In [16]:
msg = '26/1/2021'
parse_datetime(msg)

unconverted data remains: 1


-1

In [18]:
tz = pytz.timezone("Asia/Ho_Chi_Minh")
now = dt.datetime.now(tz=tz)

date_str = []
regex = REGEX_DATE
regex_day_month = REGEX_DAY_MONTH
regex_month_year = REGEX_MONTH_YEAR
pattern = re.compile("(%s|%s|%s)" % (
    regex, regex_month_year, regex_day_month), re.UNICODE)

matches = pattern.finditer(msg)
for match in matches:
    _dt = match.group(0)
    _dt = _dt.replace("/", "-").replace("|", "-").replace(":", "-").replace("tháng", "-").replace("thang", "-")
    for i in range(len(_dt.split("-"))):
        if len(_dt.split("-")[i]) == 1:
            _dt = _dt.replace(_dt.split("-")[i], "0"+_dt.split("-")[i])
    if len(_dt.split("-")) == 2:
        pos1 = _dt.split("-")[0]
        pos2 = _dt.split("-")[1]
        if 0 < int(pos1) < 32 and 0 < int(pos2) < 13:
            _dt = pos1+"-"+pos2+"-"+str(now.year)
    date_str.append(_dt)

In [19]:
date_str

['26-01-20201']

In [20]:
def get_mssv_entity(sample):
    interpreter = Interpreter.load(RASA_NLU_MODEL_PATH)
    parsed = interpreter.parse(sample)

    regex_entity = [i for i in parsed['entities'] if i['extractor'] == 'DIETClassifier']
    if len(regex_entity) == 1:
        if regex_entity[0]['entity'] == 'mssv':
            mssv = regex_entity[0]['value']
    else:
        mssv = -1
    
    return mssv

In [24]:
get_mssv_entity('akjfgjasdfj')

-1

In [32]:
def mssv_validation(mssv):

    final_result = dict()
    validation_bool = 1

    if mssv == '' or mssv == None or mssv == -1:
        validation_bool = 0
        mssv = -1
    else:
        mssv = (mssv.translate(str.maketrans('', '', string.punctuation))).replace(' ', '')
        mssv_regex = re.compile('([0-9]{2})+(52)+([0-9]{4})')
        
        if(len(mssv) != 8):
            validation_bool = 0
            mssv = -1
            
        if (not mssv_regex.match(mssv)):
            validation_bool = 0
            mssv = -1
        
    final_result['mssv'] = mssv
    final_result['validation_bool'] = validation_bool
    
    return final_result

In [34]:
mssv = 'skjdfgjsdgf'
mssv = get_mssv_entity(mssv)
valid_result = mssv_validation(mssv)
print(mssv)
print(valid_result)

-1
{'mssv': -1, 'validation_bool': 0}
